Este es el notebook de guillermo

In [ ]:
import pandas as pd

ruta = '/home/guillermo/code/niuxphrasus/Proyecto SENDA/Consolidado Estadísticas Hospitalarias 2014-2023.xlsx'
df = pd.read_excel(ruta)


In [ ]:
xls = pd.ExcelFile(ruta)
xls.sheet_names

In [ ]:
df = pd.read_excel(ruta, sheet_name='2022')

In [ ]:
df.columns


In [ ]:
df.head()

In [ ]:
df['Nombre Establecimiento'].nunique()

In [ ]:
cols_meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
glosa = "Dias Cama Disponibles"
df_hospitales = df[df["Nombre Nivel Cuidado"] == "Datos Establecimiento"]
df_glosa = df_hospitales[df_hospitales["Glosa"] == glosa]
zeros_por_hospital = df_glosa.set_index("Nombre Establecimiento")[cols_meses].eq(0).sum(axis=1)
hospitales_con_ceros = zeros_por_hospital[zeros_por_hospital > 0].sort_values(ascending=False)
hospitales_con_ceros

In [ ]:
# Hospitales unicos
# df_unico = df[['Nombre Establecimiento']].drop_duplicates()
# df_unico.to_csv('nombres_establecimientos_unicos.csv', index=False, encoding='utf-8-sig')


In [ ]:
ruta = '/home/guillermo/code/niuxphrasus/Proyecto SENDA/hospitales_unicos_2022.csv'
df2 = pd.read_csv(ruta)

In [ ]:
df2.to_csv('df2_limpio.csv', index=False, encoding='utf-8-sig')

In [ ]:
ruta = '/home/guillermo/code/niuxphrasus/Proyecto SENDA/hospitales_unicos_2021.csv'
df3 = pd.read_csv(ruta)
ruta = '/home/guillermo/code/niuxphrasus/Proyecto SENDA/hospitales_unicos_2023.csv'
df4 = pd.read_csv(ruta)
ruta = '/home/guillermo/code/niuxphrasus/Proyecto SENDA/hospitales_unicos_2024.csv'
df5 = pd.read_csv(ruta)
ruta = '/home/guillermo/code/niuxphrasus/Proyecto SENDA/hospitales_unicos_2025.csv'
df6 = pd.read_csv(ruta)

In [ ]:
df_merged = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

In [ ]:
df_merged = df_merged.iloc[:, [0]]

In [ ]:
df_merged.to_csv('df_merged.csv', index=False, encoding='utf-8-sig')

# Funcion para el preprocesado de datos en base a columnas

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def preprocesar_datos(df, target_column, numerical_features=None, categorical_features=None, test_size=0.2, random_state=42):
    """
    Función de preprocesado para entrenar un modelo.

    Parametros:
    - df: pd.DataFrame - El DataFrame con los datos.
    - target_column: str - El nombre de la columna objetivo (target).
    - numerical_features: list - Lista de nombres de columnas numéricas (opcional).
    - categorical_features: list - Lista de nombres de columnas categóricas (opcional).
    - test_size: float - Proporción del conjunto de prueba (por defecto 0.2).
    - random_state: int - Semilla para la aleatoriedad (por defecto 42).

    Retorna:
    - X_train, X_test, y_train, y_test: Los conjuntos de entrenamiento y prueba.
    """

    # Separa las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Imputación de datos faltantes
    # Si no se especifican las columnas, asignamos las numéricas y categóricas
    if numerical_features is None:
        numerical_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

    if categorical_features is None:
        categorical_features = X.select_dtypes(include=['object']).columns.tolist()

    # Preprocesamiento para columnas numéricas y categóricas
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Imputa con la media
        ('scaler', StandardScaler())  # Escala los datos numéricos
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputa con el valor más frecuente
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codifica con OneHotEncoder
    ])

    # Combina las transformaciones para las columnas numéricas y categóricas
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Divide el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Aplica el preprocesamiento al conjunto de entrenamiento y prueba
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return X_train, X_test, y_train, y_test


# Funcion para el preprocesado de datos en base a filas

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def preprocesar_datos_por_filas(df, rows_to_consider=None, test_size=0.2, random_state=42):
    """
    Función de preprocesado que realiza transformaciones para cada fila en vez de columna.

    Parametros:
    - df: pd.DataFrame - El DataFrame con los datos.
    - rows_to_consider: list - Lista de índices de filas a considerar (opcional).
    - test_size: float - Proporción del conjunto de prueba (por defecto 0.2).
    - random_state: int - Semilla para la aleatoriedad (por defecto 42).

    Retorna:
    - X_train, X_test, y_train, y_test: Los conjuntos de entrenamiento y prueba.
    """

    # Si no se especifican las filas a considerar, usa todas las filas
    if rows_to_consider is None:
        rows_to_consider = df.index.tolist()

    # Filtra el DataFrame por las filas seleccionadas
    df_filtered = df.loc[rows_to_consider]

    # Detecta automáticamente la columna objetivo (última columna en el DataFrame)
    target_column = df_filtered.columns[-1]

    # Separa las características (X) y la variable objetivo (y)
    X = df_filtered.drop(columns=[target_column])
    y = df_filtered[target_column]

    # Preprocesamiento por fila
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),  # Imputa con la media
                ('scaler', StandardScaler())  # Escala los datos numéricos
            ]), X.select_dtypes(include=['float64', 'int64']).columns.tolist()),  # Selecciona columnas numéricas automáticamente

            ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputa con el valor más frecuente
                ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codifica con OneHotEncoder
            ]), X.select_dtypes(include=['object']).columns.tolist())  # Selecciona columnas categóricas automáticamente
        ]
    )

    # Aplica el preprocesamiento por fila: el preprocesador aplica las transformaciones por cada fila
    X_processed = preprocessor.fit_transform(X)

    # Divide el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test


# Preprocesado 

In [30]:
import pandas as pd

# Cargar el archivo Excel de estadísticas hospitalarias
df_estadisticas = pd.read_excel('/home/guillermo/code/niuxphrasus/Proyecto SENDA/Consolidado Estadísticas Hospitalarias 2014-2023.xlsx')

# Cargar el archivo CSV con hospitales únicos
df_hospitales = pd.read_csv('/home/guillermo/code/niuxphrasus/Proyecto SENDA/hospitales_unicos_merge.csv')

# Verificar las primeras filas de cada uno
print(df_estadisticas.head(3))
print(df_hospitales.head(3))


   Cód. SS/SEREMI Nombre SS/SEREMI  Cód. Estab. Nombre Establecimiento  \
0               0       Datos País            0             Datos País   
1               0       Datos País            0             Datos País   
2               0       Datos País            0             Datos País   

   Cód. Nivel Cuidado Nombre Nivel Cuidado                  Glosa       Acum  \
0                   0           Datos País  Dias Cama Disponibles  9156625.0   
1                   0           Datos País     Dias Cama Ocupados  7161373.0   
2                   0           Datos País         Dias de Estada  7039263.0   

        Ene       Feb       Mar       Abr       May       Jun       Jul  \
0  759310.0  684633.0  768758.0  747010.0  772696.0  760696.0  791949.0   
1  581831.0  515403.0  597638.0  575684.0  605144.0  611365.0  646145.0   
2  567418.0  513165.0  548377.0  590199.0  574454.0  590042.0  641542.0   

        Ago       Sep       Oct       Nov       Dic  
0  788720.0  761786.0  7862

In [26]:
# Filtrar el dataframe de estadísticas para solo incluir los hospitales presentes en df_hospitales
df_filtrado_hospitales = df_estadisticas[df_estadisticas['Nombre Establecimiento'].isin(df_hospitales['Nombre Establecimiento'])]

# Ver las primeras filas del dataframe filtrado
print(df_filtrado_hospitales.head())


     Cód. SS/SEREMI Nombre SS/SEREMI  Cód. Estab.  Nombre Establecimiento  \
722               3      Antofagasta       103104  Hospital de Mejillones   
723               3      Antofagasta       103104  Hospital de Mejillones   
724               3      Antofagasta       103104  Hospital de Mejillones   
725               3      Antofagasta       103104  Hospital de Mejillones   
726               3      Antofagasta       103104  Hospital de Mejillones   

     Cód. Nivel Cuidado   Nombre Nivel Cuidado                      Glosa  \
722                   0  Datos Establecimiento      Dias Cama Disponibles   
723                   0  Datos Establecimiento         Dias Cama Ocupados   
724                   0  Datos Establecimiento             Dias de Estada   
725                   0  Datos Establecimiento  Promedio Cama Disponibles   
726                   0  Datos Establecimiento          Numero de Egresos   

       Acum    Ene    Feb    Mar    Abr    May    Jun    Jul    Ago    Sep

In [29]:
df_filtrado_hospitales['Nombre Establecimiento'].nunique()

92

In [31]:
# Función para filtrar por hospital, área y mes
def filtrar_datos_por_hospital_area_mes(df, hospital, area, mes):
    """
    Filtra el DataFrame por nombre de hospital, área y mes.

    Args:
        df (pd.DataFrame): DataFrame original.
        hospital (str): Nombre del hospital.
        area (str): Nombre del área o servicio.
        mes (str): Nombre del mes, ej. 'Enero', 'Febrero'...

    Returns:
        pd.DataFrame: DataFrame filtrado.
    """
    # Filtrar por nombre de establecimiento y área
    filtro = (
        (df['Nombre Establecimiento'] == hospital) &
        (df['Nombre Nivel Cuidado'] == area)
    )

    # Columnas que contienen el mes en su nombre
    columnas_mes = [col for col in df.columns if mes.lower() in col.lower()]

    # Seleccionar las columnas necesarias
    columnas_finales = ['Nombre Establecimiento', 'Nombre Nivel Cuidado'] + columnas_mes
    df_filtrado = df.loc[filtro, columnas_finales].copy()

    return df_filtrado


In [ ]:
# Llamar a la función usando el dataframe filtrado
resultado = filtrar_datos_por_hospital_area_mes(df, "Hospital Regional de Coihaique", "Área Cuidados Intensivos Adultos", "Mar")
print(resultado)

               Nombre Establecimiento              Nombre Nivel Cuidado  \
12330  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12331  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12332  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12333  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12334  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12335  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12336  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12337  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12338  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12339  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   
12340  Hospital Regional de Coihaique  Área Cuidados Intensivos Adultos   

          Mar  
12330  248.00  
12331  151.00  
12332  116.00  
12333    8.00  
12334   13.00  
123